# 1. Problem Introduction

College students and young adults often face the challenge of balancing grocery budgets with a desire to try new, interesting recipes. Many stick to buying staple items and then add ingredients only when trying a new recipe. Our project aims to solve the question: how can students and young adults maximize exposure to diverse, advanced recipes while staying within their grocery budget?

This project focuses on developing a model to optimize grocery purchases within a given budget, B, to maximize the number of meals or recipes that can be prepared.  We define an advanced recipe to be one with at least seven ingredients, and the minimum number of recipes to be made from a set of purchased ingredients to be seven. Since there may be multiple solutions to the recipe maximization model given a budget, we then minimize the money spent on ingredients. Solving these two problems provides the cheapest set of ingredients that will make the maximum possible amount of recipes given a budget.

# 2. A description of the approach used.
To approach this problem, we required data on individual recipes and their respective ingredients, including associated prices. We identified a dataset on Kaggle containing hundreds of recipes from around the world. For ingredient pricing, we used data from the U.S. Bureau of Labor Statistics for core items such as onions, apples, potatoes, and beef. However, as the Bureau does not track prices for all ingredients, we supplemented the data using Instacart for any untracked items.

For modeling in Gamspy, we defined two primary sets:
- **I**: A one-dimensional array containing the names of ingredients (e.g., onion, beef).
- **R**: A one-dimensional array containing unique recipe IDs.

Using these sets, we modeled the relationships between recipes and ingredients with the following parameters:
- **K**:  A binary indicator matrix over sets R and I, where K(r,i)=1 indicates that recipe r includes ingredient i
- **A**:  A positive matrix where A(r,i) specifies the quantity of ingredient i used in recipe r (e.g., recipe r uses 1 lb of potatoes).
- **C**:  Specifies the cost of each ingredient.
- **B**:  Upper bound on the total expenditure on ingredients.

The model also included the following variables:
- **X**: An indicator variable indexed by R, denoting whether recipe r is selected.
- **Z**: A variable indexed by I, representing the amount of ingredient i to purchase. Since only whole amounts can be purchased, Z is an integer variable >= 0
- **L**: A variable representing the leftover quantity of ingredient i, which is penalized in the objective function using the L1 norm. L >= 0
With these variables and sets, the relationships between recipes, ingredients, and costs were mathematically modeled using the following equations to optimize the selection of recipes while minimizing waste and adhering to budget constraints.


# Initial Data Reading and Cleaning

In [305]:
import pandas as pd
import numpy as np

df = pd.read_csv('./ingredients_refined.csv')

ingredients = np.array(df['ingredient'].str.lower().values)
price = np.array(df['price'].values)
metric = np.array(df['metric'].values)

# Initial Data Processing for Amount Matrix, the Contains Matrix, and the Cuisine Matrix

In [306]:
import json

with open('./augmented_recipes.json', 'r') as f:
    recipes = json.load(f)

matrix_recipe_ingredients = pd.DataFrame(columns=ingredients, index = [recipe['id'] for recipe in recipes])
matrix_recipe_ingredients.columns = matrix_recipe_ingredients.columns.str.strip()
matrix_recipe_contains = pd.DataFrame(columns=ingredients, index = [recipe['id'] for recipe in recipes])
matrix_recipe_contains.columns = matrix_recipe_contains.columns.str.strip()

cuisines = set()

for recipe in recipes:
    cuisines.add(recipe['cuisine'])
    for ingredient, quantity in zip(recipe["ingredients"], recipe["quantities"]):
        ingredient = ingredient.strip().lower()
        for column in matrix_recipe_ingredients.columns:
            if ingredient in column:  # Check if ingredient is a substring of the column
                matrix_recipe_ingredients.loc[recipe["id"], column] = quantity
                matrix_recipe_contains.loc[recipe["id"], column] = 1

matrix_cuisine_recipe = pd.DataFrame(columns = list(cuisines), index = [recipe['id'] for recipe in recipes])

for recipe in recipes:
    cuisine = recipe['cuisine']
    matrix_cuisine_recipe.loc[recipe["id"], cuisine] = 1


matrix_recipe_ingredients = matrix_recipe_ingredients.fillna(0)
matrix_cuisine_recipe = matrix_cuisine_recipe.fillna(0)
matrix_recipe_contains = matrix_recipe_contains.fillna(0)

/var/folders/l7/ywb8q5r53j33j1lqzbzjyshm0000gn/T/ipykernel_7368/460116402.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matrix_cuisine_recipe = matrix_cuisine_recipe.fillna(0)
/var/folders/l7/ywb8q5r53j33j1lqzbzjyshm0000gn/T/ipykernel_7368/460116402.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matrix_recipe_contains = matrix_recipe_contains.fillna(0)


# Modeling

### Set definitions

In [307]:
import gamspy as gp
import numpy as np
import gamspy.math as gpm

cont = gp.Container()
I = cont.addSet('I', records = [ingredients[idx] for idx in range(len(ingredients))], description = 'A set containing all of the valid ingredients')
R = cont.addSet('R', records = [recipe['id'] for recipe in recipes], description= 'A set containing all of the recipes that can be made')
S = cont.addSet('S', records = [1/15, 1/10, 1/5, 1/4, 1/2, 1], description="Sizes of ingredients purchasable")
Cuisines = cont.addSet('Cuisines', records = [i for i in cuisines], description = 'Different cuisines')

### Parameter Definitions

In [308]:
PPS = cont.addParameter('PPS', domain=[S], description="Price per size scalar",
                        records=np.array([[1/15], [1/10], [1/5], [1/4], [1/2], [1]]))
APS = cont.addParameter('PPS', domain=[S], description="Unit amount per size scalar",
                        records=np.array([[1/15], [1/10], [1/5], [1/4], [1/2], [1]]))

C = cont.addParameter('C', domain=[S, I], description="Cost per unit of each of the ingredients",
                      records = np.expand_dims(PPS.records['value'].to_numpy(), axis=0).T * np.tile(price, (6,1)))

A = cont.addParameter('A', domain=[R, I], description="Amount of ingredient i required in recipe r", records = np.array(matrix_recipe_ingredients))
B = cont.addParameter('B', description="Budget to purchase ingredients", records=63)

Contains = cont.addParameter('contains', domain = [R, I], description = 'Binary matrix indicating that recipe r contains ingredient i', records = np.array(matrix_recipe_contains))

### Variable Definitions

In [309]:
z = cont.addVariable('z', "integer", domain=[S, I], description="Amount of ingredient i purchased")
x = cont.addVariable('x', "binary", domain=[R], description="Indicator variable to make recipe r")
l = cont.addVariable('l', "free", domain=[I], description="Leftover ingredients after all recipes are made")
m = cont.addVariable('m', 'free', description="money spent")

### Equations

In [310]:
budget = cont.addEquation('budget', 'regular',
                        description="Constrains the total money spent on ingredients to be within the budget")
budget[:] = gp.Sum([S,I], C[S,I] * z[S,I]) <= B

ingredient_amounts = cont.addEquation('ingredient_amounts', 'regular', domain=[I],
                              description="Ensures enough ingredients are purchased to satisfy the selected recipes")
ingredient_amounts[I] = gp.Sum(R, A[R, I] * x[R]) <= gp.Sum(S, APS[S] * z[S, I])

waste = cont.addEquation('waste', 'regular', domain=[I],
                          description="Sets the leftover variable equal to the amount of unused ingredients")
waste[I] = l[I] == gp.Sum(S, APS[S] * z[S, I]) -  gp.Sum(R, A[R, I] * x[R])

tot_recipes = cont.addEquation('tot_recipes', description="Ensures enough recipes for a weeks worth of dinner can be made")
tot_recipes[:] = gp.Sum(R, x[R]) >= 7

contains_recipe = cont.addEquation('contains_recipe', description="Ensures only selecting recipes with greater than 7 ingredients", domain = [R])
contains_recipe[R] = x[R] * gp.Sum(I,Contains[R, I]) >= 7 * x[R]
money_spent = cont.addEquation('money_spent', 'regular')
money_spent[:] = m == gp.Sum([S,I], C[S,I] * z[S,I])

z.lo[S, I]= 0
l.lo[:] = 0

In [27]:
objective = gp.Sum(R, x[R]) 

recipe_optimization = cont.addModel(
    name='recipe_optimization',
    problem=gp.Problem.MIP,
    equations=cont.getEquations(),
    sense=gp.Sense.MAX,
    objective=objective
)

In [312]:
B.setRecords(100)
recipe_optimization.solve(solver='CPLEX')

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,33,280,1092,MIP,CPLEX,0.129


In [313]:
m.records

,level,marginal,lower,upper,scale
0,99.931567,0.0,-inf,inf,1.0


In [314]:
recipe_optimization.solve(options=gp.Options(time_limit= 10))
df = z.records
print(f"Amount of ingredients to purchase: {len(df[df['level'] > 0])}")
# print(f"Recipes to make {x.records[x.records['level'] > 0]}")

Amount of ingredients to purchase: 83


In [315]:
numbah = recipe_optimization.objective_value
print(numbah)

33.0


In [316]:
m.records['level']

0    99.931567
Name: level, dtype: float64

In [317]:
max_recipe_eq = cont.addEquation('max_recipe_eq', description="The maximum number of recipes that can be made given the budget")
max_recipe_eq[:] = gp.Sum(R, x[R]) == recipe_optimization.objective_value

money_optimization = cont.addModel(
    name='money_optimization',
    problem=gp.Problem.MIP,
    equations=cont.getEquations(),
    sense=gp.Sense.MIN,
    objective=m
)
money_optimization.solve()


,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,98.6845833333333,281,1092,MIP,CPLEX,0.133


# Sensitivity Analysis

## budget sensitivity

In [318]:
import seaborn as sns
import matplotlib.pyplot as plt 

amount_recipes = []
budget = []
loss = []
money_spent = []


for num in range(50, 500):
    B.setRecords(num)
    recipe_optimization.solve(solver = 'cplex', options=gp.Options(time_limit=20))
    df = x.records
    number_recipes = len(df[df['level'] > 0])
    loss.append(l.records['level'].sum())
    budget.append(num)
    amount_recipes.append(number_recipes)
    money_spent.append(m.records['level'])
    if num % 10 == 0:
        print(f'Budget = {num}')

import seaborn as sns

sns.lineplot(x = budget, y = amount_recipes)
plt.grid()


Budget = 50
Budget = 60


[MODEL - WARNING] The solve was interrupted! Solve status: ResourceInterrupt. For further information, see https://gamspy.readthedocs.io/en/latest/reference/gamspy._model.html#gamspy.SolveStatus.


Budget = 70


KeyboardInterrupt: 

## Variations of Problems

Say you want to make a recipe with tomatoes
- How can this be modeled? Adding in an equation that the amount of tomatoes has to be > 0, we can add this constraint

In [ ]:
tomomatoe_equation = cont.addEquation('tomatoe_equation')
tomomatoe_equation[:] = gp.Sum(R, A[R, 'tomatoes']) >= 0


In [30]:
tomomatoe_equation = cont.addEquation('tomatoe_equation')
tomomatoe_equation[:] = gp.Sum(R, A[R, 'tomatoes']) >= 0

B.setRecords(50)

recipe_optimization_tomatoes = cont.addModel(
    name='recipe_optimization_tomatoes',
    problem=gp.Problem.MIP,
    equations=cont.getEquations(),
    sense=gp.Sense.MAX,
    objective=objective
)
recipe_optimization_tomatoes.solve(options=gp.Options(time_limit= 10))

GamspyException: There was a compilation error. Check /var/folders/l7/ywb8q5r53j33j1lqzbzjyshm0000gn/T/tmpe14s7lfb/_9e345f22-7645-4ff9-aaf1-e699ed928827.lst for more information.

=============
Error Summary
=============
2599  tomatoe_equation .. sum(R,A(R,"tomatoes")) =g= 0;
****                 $140
**** LINE      3 INPUT       /var/folders/l7/ywb8q5r53j33j1lqzbzjyshm0000gn/T/tmpe14s7lfb/_9e345f22-7645-4ff9-aaf1-e699ed928827.gms
**** 140  Unknown symbol

**** 1 ERROR(S)   0 WARNING(S)


COMPILATION TIME     =        0.001 SECONDS      5 MB  48.3.0 71b5641f DAX-DAC


USER: Academic User                                  G240910+0003Ac-GEN
      lsoule@wisc.edu                                         GPA100283
      License for teaching and research at degree granting institutions


**** FILE SUMMARY

Input      /var/folders/l7/ywb8q5r53j33j1lqzbzjyshm0000gn/T/tmpe14s7lfb/_9e345f22-7645-4ff9-aaf1-e699ed928827.gms
Output     /var/folders/l7/ywb8q5r53j33j1lqzbzjyshm0000gn/T/tmpe14s7lfb/_

In [ ]:
recipe_optimization.solve(options=gp.Options(time_limit= 10))